In [ ]:

from threading import Lock


        
# roadID 1 or 2, direction, 

cars = [1,3,5,2,4]
directions = [2,1,2,4,3]
arrivalTimes = [10,20,30,40,50]


In [ ]:
# dbx solutions

from threading import Lock


class Foo: # print in order preferred solution 
    def __init__(self):
        self.lock1 = Lock()
        self.lock2 = Lock()
        self.lock1.acquire()
        self.lock2.acquire()
    def first(self, printFirst):
        printFirst()
        self.lock1.release()
    def second(self, printSecond):
        self.lock1.acquire()
        printSecond()
        self.lock2.release()
        # above is equivalent to 
        # with self.lock1:
        #     printSecond()
        #     self.lock2.release()
    def third(self, printThird):
        self.lock2.acquire()
        printThird()        
        # above is equivalent to 
        # with self.lock2:
        #     printThird()

class TrafficLight:
    def __init__(self):
        self.lock = Lock()
        self.roadIdGreen = 1

    def carArrived(self, carId, roadId, direction, turnGreen, crossCar):
        self.lock.acquire()
        if roadId != self.IdGreen:
            self.roadIdGreen = roadId
            turnGreen()
        crossCar()
        self.lock.release()
        # equivalent to 
        with self.lock:
            if roadId != self.IdGreen:
                self.roadIdGreen = roadId
                turnGreen()
            crossCar()
        

In [ ]:
from threading import Barrier, Lock

Event     uses Lock under the hood
Semaphore uses Lock under the hoop
acquire a lock and block until unlocked 
.acquire()
release a lock from any thread 
.release()

with some_lock:
    # do something
    
is equivalent to 
try:
    # do something
finally:
    some_lock.release()
    
    
# dont understand 
class Foo:
    def __init__(self):
        self.first_barrier = Barrier(2)
        self.second_barrier = Barrier(2)
        # pass
    
    def first(self, printFirst):
        printFirst()
        self.first_barrier.wait()
        
    def second(self, printSecond):
        self.first_barrier.wait()
        printSecond()
        self.second_barrier.wait()
        
    def third(self, PrintThird):
        self.second_barrier.wait()
        printThird()


#input = [1,2,3]
input = [1,3,2]
foo = Foo()
print(foo.first)



In [ ]:

# do undertand 
class Foo:
    def __init__(self):
        # two and three start blocked 
        self.two = Semaphore(0) # blocks if negative 
        self.three = Semaphore(0)

    def first(self, printFirst):
        printFirst()
        self.two.release()

    def second(self, printSecond):
        with self.two:
            printSecond()
            self.three.release()

    def third(self, printThird):
        with self.three:
            printThird()

class Foo:
    def __init__(self):
        self.gates = (Semaphore(0),Semaphore(0))
        
    def first(self, printFirst):
        printFirst()
        self.gates[0].release()
        
    def second(self, printSecond):
        with self.gates[0]:
            printSecond()
            self.gates[1].release()
            
    def third(self, printThird):
        with self.gates[1]:
            printThird()        
            

            
# using Event - set() -> True, clear() -> False, wait() waits until flag is True            
class Foo:
    def __init__(self):
        self.done = (Event(),Event())
        
    def first(self, printFirst):
        printFirst()
        self.done[0].set()
        
    def second(self, printSecond):
        self.done[0].wait()
        printSecond()
        self.done[1].set()
            
    def third(self, printThird):
        self.done[1].wait()
        printThird()
            
            
        

In [ ]:

multi-threaded solutions 

from concurrent import futures

class Solution:
    def crawl(self, startUrl: str, htmlParser: 'HtmlParser') -> List[str]:
        hostname = lambda url: url.split('/')[2]
        seen = {startUrl}
    
        with futures.ThreadPoolExecutor(max_workers=16) as executor:
            tasks = deque([executor.submit(htmlParser.getUrls, startUrl)])
            while tasks:
                for url in tasks.popleft().result():
                    if url not in seen and hostname(startUrl) == hostname(url):
                        seen.add(url)
                        tasks.append(executor.submit(htmlParser.getUrls, url))
        
        return list(seen)


    
        queue = Queue()
        seen = {startUrl}
        start_hostname = hostname(startUrl)
        seen_lock = threading.Lock()
        
        def worker():
            while True:
                url = queue.get()
                if url is None:
                    return

                for next_url in htmlParser.getUrls(url):
                    if next_url not in seen and hostname(next_url) == start_hostname:
                        seen_lock.acquire()
                        # Acquire lock to ensure urls are no enqueed multiple times
                        if next_url not in seen:
                            seen.add(next_url)
                            queue.put(next_url)
                        seen_lock.release()
                queue.task_done()
        
        num_workers = 8
        workers = []
        queue.put(startUrl)
        
        for i in range(num_workers):
            t = threading.Thread(target=worker)
            t.start()
            workers.append(t)
        
        # Wait until empty
        queue.join()
        
        for i in range(num_workers):
            queue.put(None)
        for t in workers:
            t.join()
        
        return list(seen)    
    
    
class FizzBuzz(object):
    def __init__(self, n):
        self.n = n
        self.fz = threading.Semaphore(0)
        self.bz = threading.Semaphore(0)
        self.fzbz = threading.Semaphore(0)
        self.num = threading.Semaphore(1)

    def fizz(self, printFizz):
        for i in range(self.n/3-self.n/15):
            self.fz.acquire()
            printFizz()
            self.num.release()

    def buzz(self, printBuzz):
        for i in range(self.n/5-self.n/15):
            self.bz.acquire()
            printBuzz()
            self.num.release()

    def fizzbuzz(self, printFizzBuzz):
        for i in range(self.n/15):
            self.fzbz.acquire()
            printFizzBuzz()
            self.num.release()
            
    def number(self, printNumber):
        for i in range(1, self.n+1):
            self.num.acquire()
            if i%3==0 and i%5 ==0:
                self.fzbz.release()
            elif i%3==0:
                self.fz.release()
            elif i%5==0:
                self.bz.release()
            else:
                printNumber(i)
                self.num.release()
        
        
class FizzBuzz:
    def __init__(self, n: int):
        self.n = n
        self.sem = Semaphore()                   # default is 1
        self.sem3 = Semaphore(0)
        self.sem5 = Semaphore(0)
        self.sem15 = Semaphore(0)

    # printFizz() outputs "fizz"
    def fizz(self, printFizz: 'Callable[[], None]') -> None:
        for i in range(3, self.n + 1, 3):
            if i % 5 != 0:
                self.sem3.acquire()
                printFizz()
                self.sem.release()
    	

    # printBuzz() outputs "buzz"
    def buzz(self, printBuzz: 'Callable[[], None]') -> None:
        for i in range(5, self.n + 1, 5):
            if i % 3 != 0:
                self.sem5.acquire()
                printBuzz()
                self.sem.release()
    	

    # printFizzBuzz() outputs "fizzbuzz"
    def fizzbuzz(self, printFizzBuzz: 'Callable[[], None]') -> None:
        for i in range(15, self.n + 1, 15):
            self.sem15.acquire()
            printFizzBuzz()
            self.sem.release()
		

    # printNumber(x) outputs "x", where x is an integer.
    def number(self, printNumber: 'Callable[[int], None]') -> None:
        for i in range(1, self.n + 1):
            self.sem.acquire()
            if i % 15 == 0:
                self.sem15.release()
            elif i % 5 == 0:
                self.sem5.release()
            elif i % 3 == 0:   
                self.sem3.release()
            else:
                printNumber(i)     
                self.sem.release()

class FizzBuzz:
    def __init__(self, n: int):
        self.done = False        
        self.n = n
        self.fSem = threading.Semaphore(0)
        self.bSem = threading.Semaphore(0)
        self.fbSem = threading.Semaphore(0)
        self.nSem = threading.Semaphore(1)

    # printFizz() outputs "fizz"
    def fizz(self, printFizz: 'Callable[[], None]') -> None:
        while True:
            self.fSem.acquire()
            if self.done: break
            printFizz()
            self.nSem.release()

    # printBuzz() outputs "buzz"
    def buzz(self, printBuzz: 'Callable[[], None]') -> None:
        while True:
            self.bSem.acquire()
            if self.done: break
            printBuzz()
            self.nSem.release()

    # printFizzBuzz() outputs "fizzbuzz"
    def fizzbuzz(self, printFizzBuzz: 'Callable[[], None]') -> None:
        while True:
            self.fbSem.acquire()
            if self.done: break
            printFizzBuzz()
            self.nSem.release()

    # printNumber(x) outputs "x", where x is an integer.
    def number(self, printNumber: 'Callable[[int], None]') -> None:
        for i in range(1, self.n+1):
            self.nSem.acquire()
            if i % 15 == 0:
                self.fbSem.release()
            elif i % 3 == 0:
                self.fSem.release()
            elif i % 5 == 0:
                self.bSem.release()
            else:
                printNumber(i)
                self.nSem.release()
        self.nSem.acquire() # needed so this thread waits to set done flag only when other threads are done with printing
        self.done = True
        self.fSem.release()  # let all other threads to break and finish
        self.bSem.release()  #
        self.fbSem.release() #
        
        
class BoundedBlockingQueue(object):
    def __init__(self, capacity: int):
        self.e = Semaphore(capacity)
        self.d = Semaphore(0)
        self.q = []

    def enqueue(self, element: int) -> None:
        self.e.acquire()
        self.q.append(element)
        self.d.release()

    def dequeue(self) -> int:
        self.d.acquire()
        element = self.q.pop(0)
        self.e.release()
        return element

    def size(self) -> int:
        return len(self.q)        
                

class BoundedBlockingQueue(object):
    def __init__(self, capacity: int):
        self.capacity = capacity
        
        self.pushing = threading.Semaphore(capacity)
        self.pulling = threading.Semaphore(0)
        self.editing = threading.Lock()
      
        self.queue = collections.deque()

    def enqueue(self, element: int) -> None:
      self.pushing.acquire()
      self.editing.acquire()
      
      self.queue.append(element)
      
      self.editing.release()
      self.pulling.release()

    def dequeue(self) -> int:
        self.pulling.acquire()
        self.editing.acquire()
        
        res = self.queue.popleft()
        
        self.editing.release()
        self.pushing.release()
        return res

    def size(self) -> int:
      return len(self.queue)

class BoundedBlockingQueue(object):

    def __init__(self, capacity: int):
        self.en, self.de = Lock(), Lock()
        self.q = deque()
        self.capacity = capacity
        self.de.acquire()

    def enqueue(self, element: int) -> None:
        self.en.acquire()
        self.q.append(element)
        if len(self.q) < self.capacity:
            self.en.release()
        if self.de.locked():
            self.de.release()

    def dequeue(self) -> int:
        self.de.acquire()
        val = self.q.popleft()
        if len(self.q):
            self.de.release()
        if val and self.en.locked():
            self.en.release()
        return val
        
    def size(self) -> int:
        return len(self.q)
    
class BoundedBlockingQueue(object):
    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.cap = capacity
        self.cur = 0
        self.queue = collections.deque()
        self.cond = threading.Condition()

    def enqueue(self, element):
        """
        :type element: int
        :rtype: void
        """
        self.cond.acquire()
        while self.cur >= self.cap:
            self.cond.wait()
        
        self.queue += element,
        self.cur += 1
        
        self.cond.notifyAll()
        self.cond.release()
		

    def dequeue(self):
        """
        :rtype: int
        """
        self.cond.acquire()
        while self.cur == 0:
            self.cond.wait()
        
        ans = self.queue.popleft()
        self.cur -= 1
        
        self.cond.notifyAll()
        self.cond.release()
        return ans
		

    def size(self):
        """
        :rtype: int
        """
        return self.cur
    
class BoundedBlockingQueue(object):
    def __init__(self, capacity: int):
        self.lock = threading.Semaphore(capacity) 
        self.capacity = capacity 
        self.queue = deque()

    def enqueue(self, element: int) -> None:
        with self.lock:
            self.queue.append(element)

    def dequeue(self) -> int:
        while not self.queue:
            time.sleep(0.1)
        return self.queue.popleft()


    def size(self) -> int:
        return len(self.queue)
    
    
        

In [ ]:

import threading
import time

mutex = threading.Lock()  # is equal to threading.Semaphore(1)

def fun1():
    while True:
        mutex.acquire()
        print(1)
        mutex.release()
        time.sleep(.5)

def fun2():
    while True:
        mutex.acquire()
        print(2)
        mutex.release()
        time.sleep(.5)

t1 = threading.Thread(target=fun1).start()
t2 = threading.Thread(target=fun2).start()

import threading
import time

mutex = threading.Lock()  # is equal to threading.Semaphore(1)

def fun1():
    while True:
        with mutex:
            print(1)
        time.sleep(.5)

def fun2():
    while True:
        with mutex:
            print(2)
        time.sleep(.5)

t1 = threading.Thread(target=fun1).start()
t2 = threading.Thread(target=fun2).start()


In [ ]:

import asyncio

def hello_world(loop):
    print('Hello World')
    loop.stop()

loop = asyncio.get_event_loop()

loop.call_soon(hello_world, loop)

loop.run_forever()
loop.close()


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
values = [2,3,4,5]
def square(n):
    return n*n
def main():
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(square, values)
    for result in results:
        print(result)
if __name__ == '__main__':
    main()

In [ ]:

import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
   'http://www.cnn.com/',
   'http://europe.wsj.com/',
   'http://www.bbc.co.uk/',
   'http://some-made-up-domain.com/']

def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    print(future_to_url)
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

In [ ]:
import threading
import queue
import random
import time
def myqueue(queue):
    while not queue.empty():
        item = queue.get()
        if item is None:
            break
        print("{} removed {} from the queue".format(threading.current_thread(), item))
        queue.task_done()
        time.sleep(2)

q = queue.Queue()
for i in range(5):
    q.put(i)
print(q)
threads = []
for i in range(4):
    thread = threading.Thread(target=myqueue, args=(q,))
    thread.start()
    threads.append(thread)
for thread in threads:
    thread.join()

In [ ]:
import collections
import heapq
import threading
import queue

q = queue.Queue()
q.put(val)
q.get()



#x = 0
def increment_global():
    global x
    x += 1
    
def taskofThread(lock):
    for i in range(0, 1000000):
        lock.acquire()
        increment_global()    
        lock.release()

def main():
    global x
    x = 0
    lock = threading.Lock()
    t1 = threading.Thread(target= taskofThread, args=(lock,))
    t2 = threading.Thread(target= taskofThread, args=(lock,))

    t1.start()
    t2.start()

    t1.join()
    t2.join()    

if __name__ == "__main__":
    for i in range(10):
        main()
        print("x = {0} after Iteration {1}".format(x,i))

In [ ]:
import collections
import heapq
import threading
class Solution:

    
    def trap(self, height):
        # fill to height of the left
        # drain down to height of fill to the right 
        volume = []
        left = 0
        for ht in height:
            left = max(left, ht)
            volume.append(left)
        volume.reverse()
        height.reverse()
        right = 0
        total_water = 0
        for index, ht in enumerate(height):
            right = max(right, ht)
            total_water += min(volume[index], right) - ht
        return total_water    
    
    
height = [0,1,0,2,1,0,1,3,2,1,2,1]                
                
answer = Solution()
print(answer.trap(height))

